# Building ML Pipelines with Kubeflow

We will have some hands-on practice with [Kubeflow Pipelines](https://www.kubeflow.org/docs/components/pipelines/overview/pipelines-overview/).  Modern ML engineering is moving towards pipeline automation for rapid iteration and experiment tracking. This is especially useful in production deployments where models need to be frequently retrained to catch trends in newer data.

Kubeflow Pipelines is one component of the [Kubeflow](https://www.kubeflow.org/) suite of tools for machine learning workflows. It is deployed on top of a Kubernetes cluster and builds an infrastructure for orchestrating ML pipelines and monitoring inputs and outputs of each component. 

We will:

* setup [Kubeflow Pipelines](https://www.kubeflow.org/docs/components/pipelines/overview/pipelines-overview/) in our local workstation
* get familiar with the Kubeflow Pipelines UI
* build pipeline components with Python and the Kubeflow Pipelines SDK
* run an ML pipeline with Kubeflow Pipelines


## Setup

We will need these tool installed in our local machine to complete the exercises:

1. Docker - platform for building and running containerized applications.  You can see installation the instructions [here](https://docs.docker.com/get-docker/). If we are using Docker for Desktop (Mac or Windows), we may need to increase the resource limits to start Kubeflow Pipelines later. We can click on the Docker icon in our Task Bar, choose `Preferences` and adjust the CPU to 4, Storage to 50GB, and the memory to at least 4GB (8GB recommended). We are not maxing out any of these limits (i.e. the slider should ideally be at the midpoint or less) since it can make our machine slow or unresponsive. The important thing is to become familiar with this Kubeflow Pipelines. 

2. kubectl - tool for running commands on Kubernetes cluster see the installation instructions [here](https://kubernetes.io/docs/tasks/tools/)

3. [kind](https://kind.sigs.k8s.io/) - a Kubernetes distribution for running local clusters using Docker. Please follow the instructions [here](https://www.kubeflow.org/docs/components/pipelines/installation/localcluster-deployment/#kind) to install kind and create a local cluster. 

4. Kubeflow Pipelines (KFP) - a platform for building and deploying portable, scalable machine learning (ML) workflows based on Docker containers. Once we've created a local cluster using `kind`, we can deploy Kubeflow Pipelines with these commands. 

```
export PIPELINE_VERSION=1.7.0
kubectl apply -k "github.com/kubeflow/pipelines/manifests/kustomize/cluster-scoped-resources?ref=$PIPELINE_VERSION&timeout=300"
kubectl wait --for condition=established --timeout=300s crd/applications.app.k8s.io
kubectl apply -k "github.com/kubeflow/pipelines/manifests/kustomize/env/platform-agnostic-pns?ref=$PIPELINE_VERSION&timeout=300"
```

We can  enter the commands above one line at a time. These will setup all the deployments and spin up the pods for the entire application. These will be found in the `kubeflow` namespace. After sending the last command, it will take a moment (around 30 minutes) for all the deployments to be ready. We can send the command `kubectl get deploy -n kubeflow` a few times to check the status. We should see all deployments with the `READY` status before you can proceed to the next section.

```
NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
cache-deployer-deployment         1/1     1            1           21h
cache-server                      1/1     1            1           21h
metadata-envoy-deployment         1/1     1            1           21h
metadata-grpc-deployment          1/1     1            1           21h
metadata-writer                   1/1     1            1           21h
minio                             1/1     1            1           21h
ml-pipeline                       1/1     1            1           21h
ml-pipeline-persistenceagent      1/1     1            1           21h
ml-pipeline-scheduledworkflow     1/1     1            1           21h
ml-pipeline-ui                    1/1     1            1           21h
ml-pipeline-viewer-crd            1/1     1            1           21h
ml-pipeline-visualizationserver   1/1     1            1           21h
mysql                             1/1     1            1           21h
workflow-controller               1/1     1            1           21h
```

When everything is ready, we can run the following command to access the `ml-pipeline-ui` service.

```
kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8080:80
```

The terminal should respond with something like this:

```
Forwarding from 127.0.0.1:8080 -> 3000
Forwarding from [::1]:8080 -> 3000
```

We can then open our browser and go to `http://localhost:8080` to see the user interface.

<img src="https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course4/week3-ungraded-labs/C4_W3_Lab_1_Intro_to_KFP/img/kfp_ui.png?raw=1" alt="kfp ui">

## Operationalizing your ML Pipelines

Generating a trained model involves executing a sequence of steps. Here is a high level overview of what these steps might look like:

<img src="https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course4/week3-ungraded-labs/C4_W3_Lab_1_Intro_to_KFP/img/highlevel.jpg?raw=1" alt="highlevel.jpg">

We can recall the very first model we ever built and more likely than not, our code then also followed a similar flow. In essence, building an ML pipeline mainly involves implementing these steps but we will need to optimize our operations to deliver value to our team. Platforms such as Kubeflow helps us to build ML pipelines that can be automated, reproducible, and easily monitored. We will see these as you build our pipeline.

### Pipeline components

The main building blocks of our ML pipeline are referred to as [components](https://www.kubeflow.org/docs/components/pipelines/overview/concepts/component/). In the context of Kubeflow, these are containerized applications that run a specific task in the pipeline. Moreover, these components generate and consume *artifacts* from other components. For example, a download task will generate a dataset artifact and this will be consumed by a data splitting task. If we go back to the simple pipeline image above and describe it using tasks and artifacts, it will look something like this:

<img src="https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course4/week3-ungraded-labs/C4_W3_Lab_1_Intro_to_KFP/img/simple_dag.jpg?raw=1" alt="img/simple_dag.jpg">

This relationship between tasks and their artifacts are what constitutes a pipeline and is also called a [directed acyclic graph (DAG)](https://en.wikipedia.org/wiki/Directed_acyclic_graph).

Kubeflow Pipelines let's us create components either by [building the component specification directly](https://www.kubeflow.org/docs/components/pipelines/sdk/component-development/#component-spec) or through [Python functions](https://www.kubeflow.org/docs/components/pipelines/sdk/python-function-components/). We will use the latter since it is more intuitive and allows for quick iteration. As we gain more experience, we can explore building the component specification directly especially if we want to use different languages other than Python.

We will begin by installing the Kubeflow Pipelines SDK. Remember to restart the runtime to load the newly installed.

In [ ]:
# Install the KFP SDK
!pip install --upgrade kfp==1.7.0

Now we will import the modules we will be using to construct the Kubeflow pipeline. We will know more what these are for in the next sections.

In [ ]:
# Import the modules you will use
import kfp

# For creating the pipeline
from kfp.v2 import dsl

# For building components
from kfp.v2.dsl import component

# Type annotations for the component artifacts
from kfp.v2.dsl import (
    Input,
    Output,
    Artifact,
    Dataset,
    Model,
    Metrics
)

We will build a pipeline to train a multi-output model trained on the [Energy Effeciency dataset from the UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency). It uses the bulding features (e.g. wall area, roof area) as inputs and has two outputs: Cooling Load and Heating Load. We will follow the five-task graph above with some slight differences in the generated artifacts.

We will now build the component to load our data into the pipeline. The code is shown below and we will discuss the syntax.

In [ ]:
@component(
    packages_to_install=["pandas", "openpyxl"],
    output_component_file="download_data_component.yaml"
)
def download_data(url:str, output_csv:Output[Dataset]):
    import pandas as pd

    # Use pandas excel reader
    df = pd.read_excel(url)
    df = df.sample(frac=1).reset_index(drop=True)
    df.to_csv(output_csv.path, index=False)

When building a component, it's good to determine first its inputs and outputs.

* The dataset we want to download is an Excel file hosted by UCI [here](https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx) and we can load that using Pandas. Instead of hardcoding the URL in our code, we can design our function to accept an *input* string parameter so we can use other URLs in case the data has been transferred. 

* For the *output*, we will want to pass the downloaded dataset to the next task (i.e. data splitting). We should assign this as an `Output` type and specify what kind of artifact it is. Kubeflow provides [several of these](https://github.com/kubeflow/pipelines/blob/master/sdk/python/kfp/v2/components/types/artifact_types.py) such as `Dataset`, `Model`, `Metrics`, etc. All artifacts are saved by Kubeflow to a storage server. For local deployments, the default will be a [MinIO](https://min.io/) server. The [path](https://github.com/kubeflow/pipelines/blob/master/sdk/python/kfp/v2/components/types/artifact_types.py#L51) property fetches the location where this artifact will be saved and that's what we did above when we called `df.to_csv(output_csv.path, index=False)`

The inputs and outputs are declared as parameters in the function definition. As we can see in the code we defined a `url` parameter with a `str` type and an `output_csv` parameter with an `Output[Dataset]` type.

Lastly, we'll need to use the `component` decorator to specify that this is a Kubeflow Pipeline component. The [documentation](https://github.com/kubeflow/pipelines/blob/master/sdk/python/kfp/v2/components/component_decorator.py#L23) shows several parameters we can set and two of them are used in the code above. As the name suggests, the `packages_to_install` argument declares any extra packages outside the base image that is needed to run our code. As of writing, the default base image is `python:3.7` so we'll need `pandas` and `openpyxl` to load the Excel file. 

The `output_component_file` is an output file that contains the specification for our newly built component. We should see it in the file explorer once we've ran the cell above. We'll see our code there and other settings that pertain to our component. We can use this file when building other pipelines if necessary. We don't have to redo our code again in a notebook in our next project as long as we have this YAML file. We can also pass this to our team members or use it in another machine. Kubeflow also hosts other reusable modules in their repo [here](https://github.com/kubeflow/pipelines/tree/master/components). For example, if we want a file downloader component in one of our projects, we can load the component from that repo using the [load_component_from_url](https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.components.html#kfp.components.ComponentStore.load_component_from_url) function as shown below. The [YAML file](https://raw.githubusercontent.com/kubeflow/pipelines/master/components/web/Download/component-sdk-v2.yaml) of that component should tell us the inputs and outputs so we can use it accordingly.

```
web_downloader_op = kfp.components.load_component_from_url(
    'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/web/Download/component-sdk-v2.yaml')
```

We will build the next component in the pipeline. Like in the previous step, we should design it first with inputs and outputs in mind. We know that the input of this component will come from the artifact generated by the `download_data()` function above. To declare input artifacts, we can annotate our parameter with the `Input[Dataset]` data type as shown below. For the outputs, we want to have two: train and test datasets. 

In [ ]:
@component(
    packages_to_install=["pandas", "scikit-learn"],
    output_component_file="split_data_component.yaml"
)
def split_data(input_csv: Input[Dataset], train_csv: Output[Dataset], test_csv: Output[Dataset]):
    import pandas as pd
    from sklearn.model_selection import train_test_split

    df = pd.read_csv(input_csv.path)
    train, test = train_test_split(df, test_size=0.2)

    train.to_csv(train_csv.path, index=False)
    test.to_csv(test_csv.path, index=False)

### Building and Running a Pipeline

Now that we have at least two components, we can try building a pipeline just to quickly see how it works. The code is shown below. Basically, we just define a function with the sequence of steps then use the `dsl.pipeline` decorator. Notice in the last line (i.e. `split_data_task`) that to get a particular artifact from a previous step, we will need to use the `outputs` dictionary and use the parameter name as the key.

In [ ]:
@dsl.pipeline(
    name="my-pipeline",
)
def my_pipeline(url: str):
    download_data_task = download_data(url=url)
    split_data_task = split_data(input_csv=download_data_task.outputs['output_csv'])

To generate our pipeline specification file, we need to compile our pipeline function using the [`Compiler`](https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.compiler.html#kfp.compiler.Compiler) class as shown below.

In [ ]:
kfp.compiler.Compiler(mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE).compile(
    pipeline_func=my_pipeline,
    package_path='pipeline.yaml')

After running the cell, we'll see a `pipeline.yaml` file in the file explorer.

We can run a pipeline programmatically or from the UI. For this exercise, you will do it from the UI and you will see how it is done programmatically in the Qwiklabs later this week. 

Please go back to the Kubeflow Pipelines UI and click `Upload Pipelines` from the `Pipelines` page.

<img src="https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course4/week3-ungraded-labs/C4_W3_Lab_1_Intro_to_KFP/img/upload.png?raw=1" alt="upload.png" width="800">
<br>
<br>

Next, select `Upload a file` and choose the `pipeline.yaml` we downloaded earlier then click `Create`. This will open a screen showing our simple DAG (just two tasks). 

<img src="https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course4/week3-ungraded-labs/C4_W3_Lab_1_Intro_to_KFP/img/dag_kfp.png?raw=1" alt="dag_kfp.png" width="640">
<br>
<br>

Click `Create Run` then scroll to the bottom to input the URL of the Excel file: https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx . Then Click `Start`.

<img src="https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course4/week3-ungraded-labs/C4_W3_Lab_1_Intro_to_KFP/img/url.png?raw=1" alt="url.png" width="640">
<br>
<br>

Select the topmost entry in the `Runs` page and we should see the progress of our run. We can click on the `download-data` box to see more details about that particular task (i.e. the URL input and the container logs). After it turns green, we should also see the output artifact and we can download it if we want by clicking the minio link. 

<img src="https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course4/week3-ungraded-labs/C4_W3_Lab_1_Intro_to_KFP/img/progress.png?raw=1" alt="progress.png" width="800">
<br>
<br>

Eventually, both tasks will turn green indicating that the run completed successfully.

### Generate the rest of the components

Now that we've seen a sample workflow, we can build the rest of the components for preprocessing, model training, and model evaluation. The functions will be longer because the task is more complex. Nonetheless, it follows the same principles as before such as declaring inputs and outputs, and specifying the additional packages.

In the `eval_model()` function, we'll notice the use of the [`log_metric()`](https://github.com/kubeflow/pipelines/blob/master/sdk/python/kfp/v2/components/types/artifact_types.py#L123) to record the results. We'll see this in the `Visualizations` tab of that task after it has completed. 

In [ ]:
@component(
    packages_to_install=["pandas", "numpy"],
    output_component_file="preprocess_data_component.yaml"
)
def preprocess_data(input_train_csv: Input[Dataset], input_test_csv: Input[Dataset], 
                    output_train_x: Output[Dataset], output_test_x: Output[Dataset],
                    output_train_y: Output[Artifact], output_test_y: Output[Artifact]):
    
    import pandas as pd
    import numpy as np
    import pickle
    
    def format_output(data):
        y1 = data.pop('Y1')
        y1 = np.array(y1)
        y2 = data.pop('Y2')
        y2 = np.array(y2)
        return y1, y2

    def norm(x, train_stats):
        return (x - train_stats['mean']) / train_stats['std']

    train = pd.read_csv(input_train_csv.path)
    test = pd.read_csv(input_test_csv.path)

    train_stats = train.describe()

    # Get Y1 and Y2 as the 2 outputs and format them as np arrays
    train_stats.pop('Y1')
    train_stats.pop('Y2')
    train_stats = train_stats.transpose()
    
    train_Y = format_output(train)
    with open(output_train_y.path, "wb") as file:
      pickle.dump(train_Y, file)
    
    test_Y = format_output(test)
    with open(output_test_y.path, "wb") as file:
      pickle.dump(test_Y, file)

    # Normalize the training and test data
    norm_train_X = norm(train, train_stats)
    norm_test_X = norm(test, train_stats)

    norm_train_X.to_csv(output_train_x.path, index=False)
    norm_test_X.to_csv(output_test_x.path, index=False)



@component(
    packages_to_install=["tensorflow", "pandas"],
    output_component_file="train_model_component.yaml"
)
def train_model(input_train_x: Input[Dataset], input_train_y: Input[Artifact], 
                output_model: Output[Model], output_history: Output[Artifact]):
    import pandas as pd
    import tensorflow as tf
    import pickle
    
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, Input
    
    norm_train_X = pd.read_csv(input_train_x.path)

    with open(input_train_y.path, "rb") as file:
        train_Y = pickle.load(file)

    def model_builder(train_X):

      # Define model layers.
      input_layer = Input(shape=(len(train_X.columns),))
      first_dense = Dense(units='128', activation='relu')(input_layer)
      second_dense = Dense(units='128', activation='relu')(first_dense)

      # Y1 output will be fed directly from the second dense
      y1_output = Dense(units='1', name='y1_output')(second_dense)
      third_dense = Dense(units='64', activation='relu')(second_dense)

      # Y2 output will come via the third dense
      y2_output = Dense(units='1', name='y2_output')(third_dense)

      # Define the model with the input layer and a list of output layers
      model = Model(inputs=input_layer, outputs=[y1_output, y2_output])

      print(model.summary())

      return model

    model = model_builder(norm_train_X)

    # Specify the optimizer, and compile the model with loss functions for both outputs
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model.compile(optimizer=optimizer,
                  loss={'y1_output': 'mse', 'y2_output': 'mse'},
                  metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                          'y2_output': tf.keras.metrics.RootMeanSquaredError()})
    # Train the model for 500 epochs
    history = model.fit(norm_train_X, train_Y, epochs=100, batch_size=10)
    model.save(output_model.path)

    with open(output_history.path, "wb") as file:
        train_Y = pickle.dump(history.history, file)



@component(
    packages_to_install=["tensorflow", "pandas"],
    output_component_file="eval_model_component.yaml"
)
def eval_model(input_model: Input[Model], input_history: Input[Artifact], 
               input_test_x: Input[Dataset], input_test_y: Input[Artifact], 
               MLPipeline_Metrics: Output[Metrics]):
    import pandas as pd
    import tensorflow as tf
    import pickle

    model = tf.keras.models.load_model(input_model.path)
    
    norm_test_X = pd.read_csv(input_test_x.path)

    with open(input_test_y.path, "rb") as file:
        test_Y = pickle.load(file)

    # Test the model and print loss and mse for both outputs
    loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_X, y=test_Y)
    print("Loss = {}, Y1_loss = {}, Y1_mse = {}, Y2_loss = {}, Y2_mse = {}".format(loss, Y1_loss, Y1_rmse, Y2_loss, Y2_rmse))
    
    MLPipeline_Metrics.log_metric("loss", loss)
    MLPipeline_Metrics.log_metric("Y1_loss", Y1_loss)
    MLPipeline_Metrics.log_metric("Y2_loss", Y2_loss)
    MLPipeline_Metrics.log_metric("Y1_rmse", Y1_rmse)
    MLPipeline_Metrics.log_metric("Y2_rmse", Y2_rmse)

### Build and run the complete pipeline

We can then build and run the entire pipeline as we did earlier. It will take around 20 minutes for all the tasks to complete and we can see the `Logs` tab of each task to see how it's going. For instance, we can see there the model training epochs as we normally see in a notebook environment.

In [ ]:
# Define a pipeline and create a task from a component:
@dsl.pipeline(
    name="my-pipeline",
)
def my_pipeline(url: str):
    
    download_data_task = download_data(url=url)
    
    split_data_task = split_data(input_csv=download_data_task.outputs['output_csv'])
    
    preprocess_data_task = preprocess_data(input_train_csv=split_data_task.outputs['train_csv'],
                                           input_test_csv=split_data_task.outputs['test_csv'])
    
    train_model_task = train_model(input_train_x=preprocess_data_task.outputs["output_train_x"],
                                   input_train_y=preprocess_data_task.outputs["output_train_y"])
    
    eval_model_task = eval_model(input_model=train_model_task.outputs["output_model"],
                                 input_history=train_model_task.outputs["output_history"],
                                   input_test_x=preprocess_data_task.outputs["output_test_x"],
                                   input_test_y=preprocess_data_task.outputs["output_test_y"])

In [ ]:
kfp.compiler.Compiler(mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE).compile(
    pipeline_func=my_pipeline,
    package_path='pipeline.yaml')

After we've uploaded and ran the entire pipeline, we should see all green boxes and the training metrics in the `Visualizations` tab of the `eval-model` task.

<img src='https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/blob/main/course4/week3-ungraded-labs/C4_W3_Lab_1_Intro_to_KFP/img/complete_pipeline.png?raw=1' alt="./img/complete_pipeline.png" width=640>

## Tear Down

If we're done experimenting with the software and want to free up resources, we can execute the commands below to delete Kubeflow Pipelines from our system:

```
export PIPELINE_VERSION=1.7.0
kubectl delete -k "github.com/kubeflow/pipelines/manifests/kustomize/env/platform-agnostic-pns?ref=$PIPELINE_VERSION"
kubectl delete -k "github.com/kubeflow/pipelines/manifests/kustomize/cluster-scoped-resources?ref=$PIPELINE_VERSION"
```

we can delete the cluster for `kind` with the following:
```
kind delete cluster
```